In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml("mnist_784" ,version=1)

In [3]:
X = mnist["data"]
y = mnist["target"]

In [4]:
y = y.astype(np.uint8) # converting string labels to integers

In [5]:
X_train, X_test, y_train, y_test = X.iloc[:60000], X.iloc[60000:], y.iloc[:60000], y.iloc[60000:]

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

svm_clf = SVC()
param_grid = [{"kernel": ["linear", "rbf"],
              "C": [1.0, 100.0, 500.0],
              "gamma": ["scale", "auto"]
              }]

grid_cv = GridSearchCV(svm_clf, param_grid, cv=3, scoring="accuracy")
grid_cv.fit(X_train_scaled, y_train)

In [ ]:
grid_cv.best_params_

In [ ]:
grid_cv.best_estimator_